In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import json
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bidding_imbalance_price = pd.read_csv('bidding_training_predictions_imbalance_price.csv')
df_bidding_imbalance_price = df_bidding_imbalance_price[["timestamp_utc","imbalance_price_predictions"]]
df_bidding_imbalance_price.timestamp_utc = pd.to_datetime(df_bidding_imbalance_price.timestamp_utc)
df_bidding_day_ahead_price = pd.read_csv('bidding_training_predictions_day_ahead_price.csv')
df_bidding_day_ahead_price = df_bidding_day_ahead_price[["timestamp_utc","day_ahead_price_predictions"]]
df_bidding_day_ahead_price.timestamp_utc = pd.to_datetime(df_bidding_day_ahead_price.timestamp_utc)
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
# df_day_ahead = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/day_ahead_price.csv')
# df_imbalance = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/imbalance_price.csv')

In [3]:
df_bbidding.timestamp_utc = pd.to_datetime(df_bbidding.timestamp_utc)
df_bbidding = df_bbidding.merge(df_bidding_imbalance_price, on="timestamp_utc")
df_bbidding = df_bbidding.merge(df_bidding_day_ahead_price, on="timestamp_utc")

In [4]:
df_bbidding.Target_MW

0        102.844285
1        162.689737
2        172.662460
3        174.278554
4        206.460381
            ...    
41944     23.648000
41945     10.865000
41946     10.505000
41947      4.486000
41948      6.580000
Name: Target_MW, Length: 41949, dtype: float64

50% quantil gibt historisch 25437.310022730526 pro periode

In [5]:
df_bbidding["Revenue_40"] = df_bbidding["day_ahead_price"] * df_bbidding["4"]+(df_bbidding["Target_MW"]-df_bbidding["4"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["4"]))
df_bbidding["Revenue_30"] = df_bbidding["day_ahead_price"] * df_bbidding["3"]+(df_bbidding["Target_MW"]-df_bbidding["3"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["3"]))
df_bbidding["Revenue_20"] = df_bbidding["day_ahead_price"] * df_bbidding["2"]+(df_bbidding["Target_MW"]-df_bbidding["2"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["2"]))
df_bbidding["Revenue_10"] = df_bbidding["day_ahead_price"] * df_bbidding["1"]+(df_bbidding["Target_MW"]-df_bbidding["1"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["1"]))
df_bbidding["Revenue_60"] = df_bbidding["day_ahead_price"] * df_bbidding["6"]+(df_bbidding["Target_MW"]-df_bbidding["6"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["6"]))
df_bbidding["Revenue_50"] = df_bbidding["day_ahead_price"] * df_bbidding["5"]+(df_bbidding["Target_MW"]-df_bbidding["5"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["5"]))
df_bbidding["Revenue_70"] = df_bbidding["day_ahead_price"] * df_bbidding["7"]+(df_bbidding["Target_MW"]-df_bbidding["7"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["7"]))
df_bbidding["Revenue_80"] = df_bbidding["day_ahead_price"] * df_bbidding["8"]+(df_bbidding["Target_MW"]-df_bbidding["8"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["8"]))
df_bbidding["Revenue_90"] = df_bbidding["day_ahead_price"] * df_bbidding["9"]+(df_bbidding["Target_MW"]-df_bbidding["9"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["9"]))

In [6]:
for i in range(1,10):
    print("Revenue_"+str(i)+"0")
    print(df_bbidding["Revenue_"+str(i)+"0"].mean())

Revenue_10
36424.5882388013
Revenue_20
36525.27897374193
Revenue_30
36075.702761074645
Revenue_40
35064.55656388621
Revenue_50
35100.56368650106
Revenue_60
34516.787675207015
Revenue_70
34090.41254056421
Revenue_80
33174.758811071966
Revenue_90
30826.51927341632


In [7]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price']
    Target_MW = row['Target_MW']
    imbalance_price = row['imbalance_price']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_optimal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))



Mit optimierung 56488.5046925057 pro periode

In [8]:
df_bbidding.revenue_optimal.mean()

56847.65672443437

In [9]:
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

Revenue mit estimateten values (LSTM) Mit 50% quantile

In [10]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['5']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))


In [11]:
df_bbidding.revenue_normal.mean()

31904.5633050109

LSTM 40% quantile

In [12]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['1']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))


In [13]:
df_bbidding.revenue_normal.mean()

33951.44619676805

## On newer data

In [14]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [15]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [16]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)


In [17]:
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new.timestamp = pd.to_datetime(df_api_new.timestamp)

In [18]:

# Create a continuous time series from the minimum to maximum timestamp at 30-minute intervals
full_timestamp_range = pd.date_range(start=df_api_new['timestamp'].min(), end=df_api_new['timestamp'].max(), freq='30min')
# Reindex the dataframe using the full range of timestamps
df_api_new_1 = df_api_new.set_index('timestamp').reindex(full_timestamp_range, method=None)
df_api_new_1 = df_api_new_1.reset_index().rename(columns={'index': 'timestamp'})
# Create the 'prediction_date' column based on the timestamp
df_api_new_1['prediction_date'] = df_api_new_1['timestamp'].dt.date

In [19]:
df_api_new_1

,timestamp,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-10-03 22:00:00+00:00,2024-10-03,1355.0,487.0,963.0,1544.0,1330.0,867.0,326.0,1402.0,1357.0
1,2024-10-03 22:30:00+00:00,2024-10-03,788.0,584.0,1148.0,1320.0,1007.0,565.0,923.0,797.0,487.0
2,2024-10-03 23:00:00+00:00,2024-10-03,571.0,748.0,1001.0,1460.0,740.0,1017.0,1533.0,597.0,991.0
3,2024-10-03 23:30:00+00:00,2024-10-03,1476.0,1194.0,1512.0,334.0,1054.0,1572.0,744.0,1497.0,463.0
4,2024-10-04 00:00:00+00:00,2024-10-04,1352.0,1390.0,912.0,662.0,1505.0,1093.0,814.0,487.0,1340.0
...,...,...,...,...,...,...,...,...,...,...,...
1051,2024-10-25 19:30:00+00:00,2024-10-25,0.0,26.0,57.0,83.0,107.0,137.0,160.0,197.0,274.0
1052,2024-10-25 20:00:00+00:00,2024-10-25,0.0,34.0,68.0,95.0,125.0,152.0,175.0,221.0,313.0
1053,2024-10-25 20:30:00+00:00,2024-10-25,0.0,27.0,58.0,85.0,115.0,148.0,173.0,211.0,282.0
1054,2024-10-25 21:00:00+00:00,2024-10-25,0.0,16.0,49.0,76.0,105.0,139.0,162.0,201.0,266.0


In [20]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new_1,df_day_ahead_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.day/7)


In [21]:
df_api_new_merged1 = df_api_new_merged[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(192)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(96)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1 = df_api_new_merged1.groupby("timestamp_utc").last().reset_index()


In [22]:
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')
wind_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/wind_total_production.csv')
solar_total.generation_mw = solar_total.generation_mw * 0.5
wind_total.generation_mw = wind_total.generation_mw * 0.5 - wind_total.boa
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
wind_total.timestamp_utc = pd.to_datetime(wind_total.timestamp_utc)

In [23]:
df_api_new_merged2 = pd.merge(df_api_new_merged1,solar_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = pd.merge(df_api_new_merged2,wind_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = df_api_new_merged2.rename(columns={
    "generation_mw_x": "generation_solar",
    "generation_mw_y": "generation_wind"
})
df_api_new_merged2 = df_api_new_merged2.groupby("timestamp_utc").last().reset_index()
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h,generation_solar,installed_capacity_mwp,capacity_mwp,settlement_date,settlement_period_y,boa,generation_wind
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,61.00,78.47,1627.05,0.0,2956.745251,2779.542388,2024-10-07,47,0.0,236.742
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,50.00,78.47,1469.90,0.0,2956.745251,2779.542388,2024-10-07,48,0.0,154.032
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,93.50,71.36,1405.55,0.0,2956.745251,2779.542388,2024-10-08,1,0.0,124.472
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,93.50,71.36,1268.95,0.0,2956.745251,2779.477654,2024-10-08,2,0.0,145.582
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,93.00,72.80,1428.80,0.0,2956.745251,2779.477632,2024-10-08,3,0.0,160.762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2024-10-16 19:30:00+00:00,86.52,84.34,1816.40,42.0,0.258819,-0.222521,1001.0,1087.0,1107.0,...,-20.63,58.49,318.15,0.0,2956.745251,2778.959091,2024-10-16,42,0.0,586.101
377,2024-10-16 20:00:00+00:00,86.24,79.89,1522.30,43.0,0.500000,-0.222521,1003.0,1099.0,1111.0,...,23.45,28.69,289.25,0.0,2956.745251,2778.959067,2024-10-16,43,0.0,585.848
378,2024-10-16 20:30:00+00:00,88.87,79.89,1911.35,44.0,0.500000,-0.222521,995.0,1090.0,1109.0,...,25.00,28.69,541.20,0.0,2956.745251,2778.959055,2024-10-16,44,0.0,585.851
379,2024-10-16 21:00:00+00:00,88.20,83.31,1651.15,45.0,0.707107,-0.222521,1031.0,1083.0,1110.0,...,25.00,0.68,1282.20,0.0,2956.745251,2778.959045,2024-10-16,45,0.0,585.931


In [24]:
import pickle
df_api_new_merged2_X = df_api_new_merged2[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].copy()

scaler_path = "LSTM_imbalance_scaler.pkl"
# Laden des StandardScalers aus der Datei
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

# Skalieren der Daten
df_api_new_merged2_X_scaled = scaler.transform(df_api_new_merged2_X)

# Konvertieren der Daten in PyTorch-Tensoren
X_test = torch.tensor(df_api_new_merged2_X_scaled, dtype=torch.float32)
X_test = X_test.unsqueeze(1)  # Adds a sequence length dimension

c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [25]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_imbalance_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["imvalance_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_11040\1576695596.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_imbalance_p

In [26]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["day_ahead_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_11040\823899303.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_pr

In [27]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,volume_lag96h,generation_solar,installed_capacity_mwp,capacity_mwp,settlement_date,settlement_period_y,boa,generation_wind,imvalance_price_predictions,day_ahead_price_predictions
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,1627.05,0.0,2956.745251,2779.542388,2024-10-07,47,0.0,236.742,79.676735,1.275775
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,1469.90,0.0,2956.745251,2779.542388,2024-10-07,48,0.0,154.032,78.965164,1.437075
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,1405.55,0.0,2956.745251,2779.542388,2024-10-08,1,0.0,124.472,84.008194,1.385978
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,1268.95,0.0,2956.745251,2779.477654,2024-10-08,2,0.0,145.582,80.414536,1.463478
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,1428.80,0.0,2956.745251,2779.477632,2024-10-08,3,0.0,160.762,73.853539,-0.223743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2024-10-16 19:30:00+00:00,86.52,84.34,1816.40,42.0,0.258819,-0.222521,1001.0,1087.0,1107.0,...,318.15,0.0,2956.745251,2778.959091,2024-10-16,42,0.0,586.101,69.862236,-9.821907
377,2024-10-16 20:00:00+00:00,86.24,79.89,1522.30,43.0,0.500000,-0.222521,1003.0,1099.0,1111.0,...,289.25,0.0,2956.745251,2778.959067,2024-10-16,43,0.0,585.848,64.894478,-9.899858
378,2024-10-16 20:30:00+00:00,88.87,79.89,1911.35,44.0,0.500000,-0.222521,995.0,1090.0,1109.0,...,541.20,0.0,2956.745251,2778.959055,2024-10-16,44,0.0,585.851,63.363705,-9.903649
379,2024-10-16 21:00:00+00:00,88.20,83.31,1651.15,45.0,0.707107,-0.222521,1031.0,1083.0,1110.0,...,1282.20,0.0,2956.745251,2778.959045,2024-10-16,45,0.0,585.931,59.906662,-9.883387


In [28]:
df_api_new_merged2["Total_MW"] = df_api_new_merged2["generation_solar"] + df_api_new_merged2["generation_wind"]

In [29]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,generation_solar,installed_capacity_mwp,capacity_mwp,settlement_date,settlement_period_y,boa,generation_wind,imvalance_price_predictions,day_ahead_price_predictions,Total_MW
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,0.0,2956.745251,2779.542388,2024-10-07,47,0.0,236.742,79.676735,1.275775,236.742
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,0.0,2956.745251,2779.542388,2024-10-07,48,0.0,154.032,78.965164,1.437075,154.032
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,0.0,2956.745251,2779.542388,2024-10-08,1,0.0,124.472,84.008194,1.385978,124.472
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,0.0,2956.745251,2779.477654,2024-10-08,2,0.0,145.582,80.414536,1.463478,145.582
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,0.0,2956.745251,2779.477632,2024-10-08,3,0.0,160.762,73.853539,-0.223743,160.762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2024-10-16 19:30:00+00:00,86.52,84.34,1816.40,42.0,0.258819,-0.222521,1001.0,1087.0,1107.0,...,0.0,2956.745251,2778.959091,2024-10-16,42,0.0,586.101,69.862236,-9.821907,586.101
377,2024-10-16 20:00:00+00:00,86.24,79.89,1522.30,43.0,0.500000,-0.222521,1003.0,1099.0,1111.0,...,0.0,2956.745251,2778.959067,2024-10-16,43,0.0,585.848,64.894478,-9.899858,585.848
378,2024-10-16 20:30:00+00:00,88.87,79.89,1911.35,44.0,0.500000,-0.222521,995.0,1090.0,1109.0,...,0.0,2956.745251,2778.959055,2024-10-16,44,0.0,585.851,63.363705,-9.903649,585.851
379,2024-10-16 21:00:00+00:00,88.20,83.31,1651.15,45.0,0.707107,-0.222521,1031.0,1083.0,1110.0,...,0.0,2956.745251,2778.959045,2024-10-16,45,0.0,585.931,59.906662,-9.883387,585.931


In [30]:
df_api_new_merged2["Revenue_40"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["4"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["4"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["4"]))
df_api_new_merged2["Revenue_30"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["3"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["3"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["3"]))
df_api_new_merged2["Revenue_20"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["2"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["2"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["2"]))
df_api_new_merged2["Revenue_10"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["1"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["1"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["1"]))
df_api_new_merged2["Revenue_60"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["6"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["6"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["6"]))
df_api_new_merged2["Revenue_50"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["5"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["5"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["5"]))
df_api_new_merged2["Revenue_70"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["7"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["7"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["7"]))
df_api_new_merged2["Revenue_80"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["8"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["8"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["8"]))
df_api_new_merged2["Revenue_90"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["9"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["9"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["9"]))

In [31]:
for i in range(1,10):
    print("Revenue_"+str(i)+"0")
    print(df_api_new_merged2["Revenue_"+str(i)+"0"].mean())

Revenue_10
18815.080086303027
Revenue_20
15932.990505146156
Revenue_30
14106.68994360952
Revenue_40
12439.874967047537
Revenue_50
10654.882516248177
Revenue_60
9018.052752643613
Revenue_70
7133.450060386359
Revenue_80
4981.380272183181
Revenue_90
2648.4172137404594


In [32]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['5']
    imbalance_price = row['imvalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']))


In [33]:
df_api_new_merged2.revenue_normal.mean()

22094.32860787391

In [34]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['1']
    imbalance_price = row['imvalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']))


In [35]:
df_api_new_merged2.revenue_normal.mean()

22379.32269403627

In [36]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,Revenue_30,Revenue_20,Revenue_10,Revenue_60,Revenue_50,Revenue_70,Revenue_80,Revenue_90,optimized_trade,revenue_normal
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,20075.583981,20116.491741,20156.839501,19927.996821,19970.864581,19884.569061,19818.377421,19773.549661,0.000000,19750.925781
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,9166.269208,9252.018168,9337.207128,8861.737848,8949.446808,8773.468888,8640.015448,8550.346488,0.000000,8505.302008
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,7693.513685,7766.185845,7838.298005,7434.751125,7509.383285,7321.752885,7207.494645,7169.128565,0.000000,7130.622485
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,8732.045889,8810.628849,8888.651809,8452.595529,8533.138489,8330.731089,8207.606649,8166.285169,0.000000,8124.823689
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,9166.634075,9239.567435,9311.940795,8906.957315,9019.087355,8831.503955,8755.490595,8678.917235,0.000000,8640.420555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2024-10-16 19:30:00+00:00,86.52,84.34,1816.40,42.0,0.258819,-0.222521,1001.0,1087.0,1107.0,...,38038.170976,39176.888176,43435.252136,35400.298996,35720.328296,34880.972116,34219.213516,34152.267656,431.826955,45514.866349
377,2024-10-16 20:00:00+00:00,86.24,79.89,1522.30,43.0,0.500000,-0.222521,1003.0,1099.0,1111.0,...,14311.906383,15485.401743,24147.604623,10462.837743,10877.522863,9416.324943,8462.493423,8355.812143,468.753019,48783.865276
378,2024-10-16 20:30:00+00:00,88.87,79.89,1911.35,44.0,0.500000,-0.222521,995.0,1090.0,1109.0,...,16078.870666,17865.267006,26039.148706,11747.912826,13254.825726,10725.804226,10416.441646,10001.998206,471.659605,48415.630902
379,2024-10-16 21:00:00+00:00,88.20,83.31,1651.15,45.0,0.707107,-0.222521,1031.0,1083.0,1110.0,...,19793.679827,22228.260647,26629.522967,16096.883087,16485.841727,15213.536147,14218.739547,13917.570567,532.499464,49612.701985


MLPs mit extra dmeand und margin

In [37]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        # Layer sizes from the best trial
        layer_sizes = [256, 448, 192, 96]
        dropout_rates = [0.12338360578207397, 0.2192742565593194, 0.15708417985889997, 0.253419888887539]

        # Define the layers
        self.fc1 = nn.Linear(input_dim, layer_sizes[0])
        self.fc2 = nn.Linear(layer_sizes[0], layer_sizes[1])
        self.fc3 = nn.Linear(layer_sizes[1], layer_sizes[2])
        self.fc4 = nn.Linear(layer_sizes[2], layer_sizes[3])
        self.fc5 = nn.Linear(layer_sizes[3], 1)  # Output layer

        # Dropouts
        self.dropout1 = nn.Dropout(dropout_rates[0])
        self.dropout2 = nn.Dropout(dropout_rates[1])
        self.dropout3 = nn.Dropout(dropout_rates[2])
        self.dropout4 = nn.Dropout(dropout_rates[3])

        # Activation function (Swish)
        self.swish = nn.SiLU()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # First layer with ReLU
        x = self.dropout1(x)         # First dropout
        x = self.swish(self.fc2(x))  # Second layer with Swish
        x = self.dropout2(x)         # Second dropout
        x = self.swish(self.fc3(x))  # Third layer with Swish
        x = self.dropout3(x)         # Third dropout
        x = self.swish(self.fc4(x))  # Fourth layer with Swish
        x = self.dropout4(x)         # Fourth dropout
        x = self.fc5(x)              # Output layer (no activation for raw outputs)
        return x



In [38]:
import pickle
df_api_new_merged2_X = df_api_new_merged2[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].copy()

scaler_path = "LSTM_imbalance_scaler.pkl"
# Laden des StandardScalers aus der Datei
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

# Skalieren der Daten
df_api_new_merged2_X_scaled = scaler.transform(df_api_new_merged2_X)

# Konvertieren der Daten in PyTorch-Tensoren
X_test = torch.tensor(df_api_new_merged2_X_scaled, dtype=torch.float32)


c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [39]:
df_api_new_merged2.columns

Index(['timestamp_utc', 'market_price', 'day_ahead_price', 'volume',
       'settlement_period_x', 'cos_hour', 'cos_day', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'imbalance_price', 'market_price_lag96h',
       'imbalance_price_lag96h', 'day_ahead_price_lag1week', 'volume_lag96h',
       'generation_solar', 'installed_capacity_mwp', 'capacity_mwp',
       'settlement_date', 'settlement_period_y', 'boa', 'generation_wind',
       'imvalance_price_predictions', 'day_ahead_price_predictions',
       'Total_MW', 'Revenue_40', 'Revenue_30', 'Revenue_20', 'Revenue_10',
       'Revenue_60', 'Revenue_50', 'Revenue_70', 'Revenue_80', 'Revenue_90',
       'optimized_trade', 'revenue_normal'],
      dtype='object')

In [40]:
model_imbalance_mlp = MLP(input_dim=15)
model_imbalance_mlp.load_state_dict(torch.load("MLP_day_ahead_price.pth"))
model_imbalance_mlp.eval()
with torch.no_grad():
    predictions = model_imbalance_mlp(X_test)
predictions = predictions.numpy()
df_api_new_merged2["day_ahead_price_predictions_MLP"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_11040\1782585997.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance_mlp.load_state_dict(torch.load("MLP_day_ahea

In [41]:
model_day_ahead_price = MLP(input_dim=15)
model_day_ahead_price.load_state_dict(torch.load("MLP_imbalance_price.pth"))
model_day_ahead_price.eval()
with torch.no_grad():
    predictions = model_day_ahead_price(X_test)
predictions = predictions.numpy()
df_api_new_merged2["imbalance_price_predictions_MLP"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_11040\360064670.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_day_ahead_price.load_state_dict(torch.load("MLP_imbalan

In [42]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions_MLP']
    Target_MW = row['5']
    imbalance_price = row['imbalance_price_predictions_MLP']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade_MLP'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade_MLP'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_MLP']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_MLP']))


In [43]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,Revenue_60,Revenue_50,Revenue_70,Revenue_80,Revenue_90,optimized_trade,revenue_normal,day_ahead_price_predictions_MLP,imbalance_price_predictions_MLP,optimized_trade_MLP
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,19927.996821,19970.864581,19884.569061,19818.377421,19773.549661,0.000000,19750.974364,101.358147,102.757851,0.002141
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,8861.737848,8949.446808,8773.468888,8640.015448,8550.346488,0.000000,8818.247083,101.848892,102.267067,7.013000
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,7434.751125,7509.383285,7321.752885,7207.494645,7169.128565,0.000000,7130.622485,96.739403,99.140266,0.000000
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,8452.595529,8533.138489,8330.731089,8207.606649,8166.285169,0.000000,8124.823689,95.311623,97.323784,0.000000
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,8906.957315,9019.087355,8831.503955,8755.490595,8678.917235,0.000000,8640.420555,98.392189,101.681694,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2024-10-16 19:30:00+00:00,86.52,84.34,1816.40,42.0,0.258819,-0.222521,1001.0,1087.0,1107.0,...,35400.298996,35720.328296,34880.972116,34219.213516,34152.267656,431.826955,35180.885418,83.921593,82.745903,1153.396830
377,2024-10-16 20:00:00+00:00,86.24,79.89,1522.30,43.0,0.500000,-0.222521,1003.0,1099.0,1111.0,...,10462.837743,10877.522863,9416.324943,8462.493423,8355.812143,468.753019,9687.469248,70.159279,68.561203,1156.421867
378,2024-10-16 20:30:00+00:00,88.87,79.89,1911.35,44.0,0.500000,-0.222521,995.0,1090.0,1109.0,...,11747.912826,13254.825726,10725.804226,10416.441646,10001.998206,471.659605,12299.563166,65.608864,64.272079,1147.545083
379,2024-10-16 21:00:00+00:00,88.20,83.31,1651.15,45.0,0.707107,-0.222521,1031.0,1083.0,1110.0,...,16096.883087,16485.841727,15213.536147,14218.739547,13917.570567,532.499464,11047.344357,54.406502,46.848373,1198.986182


In [44]:
df_api_new_merged2.revenue_normal.mean()

10326.03691494122

Better MLP

In [45]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self, input_dim):
        super(SimpleModel, self).__init__()
        
        # Zwei Hidden Layers
        self.fc1 = nn.Linear(input_dim, 128)  # Erster Hidden Layer
        self.fc2 = nn.Linear(128, 64)         # Zweiter Hidden Layer
        
        # Ausgangsschicht
        self.fc3 = nn.Linear(64, 1)           # Ausgangsschicht
        
        # Dropout und Aktivierungsfunktion
        self.dropout = nn.Dropout(0.2)
        self.swish = nn.SiLU()

    def forward(self, x):
        # Durch die Hidden Layers
        x = self.swish(self.fc1(x))  # Erster Hidden Layer
        x = self.dropout(x)
        x = self.swish(self.fc2(x))  # Zweiter Hidden Layer
        x = self.dropout(x)
        
        # Ausgang
        x = self.fc3(x)  # Ausgangsschicht
        
        return x


In [46]:
df_day_ahead_demand = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/day_ahead_demand_forecast.csv')
df_margin_forecast = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/margin_forecast.csv')
df_margin_forecast.forecast_date = pd.to_datetime(df_margin_forecast.forecast_date)
df_day_ahead_demand.timestamp_utc = pd.to_datetime(df_day_ahead_demand.timestamp_utc)

In [47]:
df_api_new_merged3 = pd.merge(df_api_new_merged2,df_day_ahead_demand, left_on='timestamp_utc', right_on='timestamp_utc', how='left')
df_api_new_merged3["date"] = df_api_new_merged3["timestamp_utc"].dt.date
df_api_new_merged3["date"] = pd.to_datetime(df_api_new_merged3["date"])
df_api_new_merged3 = pd.merge(df_api_new_merged3,df_margin_forecast, left_on='date', right_on='forecast_date', how='left')

In [48]:
df_api_new_merged3.dropna(inplace=True)
df_api_new_merged3 = df_api_new_merged3.groupby("timestamp_utc").last().reset_index()
df_api_new_merged3

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,settlement_date_y,settlement_period,boundary,publish_time_utc_x,transmission_system_demand,national_demand,date,forecast_date,publish_time_utc_y,margin
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,2024-10-07,47,N,2024-10-07T20:45:00Z,25216.0,24297.0,2024-10-07,2024-10-07,2024-10-05T21:00:00Z,8918.0
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,2024-10-07,48,N,2024-10-07T20:45:00Z,23960.0,23100.0,2024-10-07,2024-10-07,2024-10-05T21:00:00Z,8918.0
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,2024-10-08,1,N,2024-10-07T20:45:00Z,23214.0,22480.0,2024-10-07,2024-10-07,2024-10-05T21:00:00Z,8918.0
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,2024-10-08,2,N,2024-10-07T20:45:00Z,22858.0,22146.0,2024-10-07,2024-10-07,2024-10-05T21:00:00Z,8918.0
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,2024-10-08,3,N,2024-10-07T20:45:00Z,22720.0,21970.0,2024-10-08,2024-10-08,2024-10-06T21:00:00Z,10251.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2024-10-16 19:30:00+00:00,86.52,84.34,1816.40,42.0,0.258819,-0.222521,1001.0,1087.0,1107.0,...,2024-10-16,42,N,2024-10-16T17:47:00Z,33741.0,32261.0,2024-10-16,2024-10-16,2024-10-14T21:00:00Z,13209.0
377,2024-10-16 20:00:00+00:00,86.24,79.89,1522.30,43.0,0.500000,-0.222521,1003.0,1099.0,1111.0,...,2024-10-16,43,N,2024-10-16T17:47:00Z,32481.0,31001.0,2024-10-16,2024-10-16,2024-10-14T21:00:00Z,13209.0
378,2024-10-16 20:30:00+00:00,88.87,79.89,1911.35,44.0,0.500000,-0.222521,995.0,1090.0,1109.0,...,2024-10-16,44,N,2024-10-16T17:47:00Z,30889.0,29409.0,2024-10-16,2024-10-16,2024-10-14T21:00:00Z,13209.0
379,2024-10-16 21:00:00+00:00,88.20,83.31,1651.15,45.0,0.707107,-0.222521,1031.0,1083.0,1110.0,...,2024-10-16,45,N,2024-10-16T20:46:00Z,28982.0,27532.0,2024-10-16,2024-10-16,2024-10-14T21:00:00Z,13209.0


In [49]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,Revenue_60,Revenue_50,Revenue_70,Revenue_80,Revenue_90,optimized_trade,revenue_normal,day_ahead_price_predictions_MLP,imbalance_price_predictions_MLP,optimized_trade_MLP
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,19927.996821,19970.864581,19884.569061,19818.377421,19773.549661,0.000000,19750.974364,101.358147,102.757851,0.002141
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,8861.737848,8949.446808,8773.468888,8640.015448,8550.346488,0.000000,8818.247083,101.848892,102.267067,7.013000
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,7434.751125,7509.383285,7321.752885,7207.494645,7169.128565,0.000000,7130.622485,96.739403,99.140266,0.000000
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,8452.595529,8533.138489,8330.731089,8207.606649,8166.285169,0.000000,8124.823689,95.311623,97.323784,0.000000
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,8906.957315,9019.087355,8831.503955,8755.490595,8678.917235,0.000000,8640.420555,98.392189,101.681694,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2024-10-16 19:30:00+00:00,86.52,84.34,1816.40,42.0,0.258819,-0.222521,1001.0,1087.0,1107.0,...,35400.298996,35720.328296,34880.972116,34219.213516,34152.267656,431.826955,35180.885418,83.921593,82.745903,1153.396830
377,2024-10-16 20:00:00+00:00,86.24,79.89,1522.30,43.0,0.500000,-0.222521,1003.0,1099.0,1111.0,...,10462.837743,10877.522863,9416.324943,8462.493423,8355.812143,468.753019,9687.469248,70.159279,68.561203,1156.421867
378,2024-10-16 20:30:00+00:00,88.87,79.89,1911.35,44.0,0.500000,-0.222521,995.0,1090.0,1109.0,...,11747.912826,13254.825726,10725.804226,10416.441646,10001.998206,471.659605,12299.563166,65.608864,64.272079,1147.545083
379,2024-10-16 21:00:00+00:00,88.20,83.31,1651.15,45.0,0.707107,-0.222521,1031.0,1083.0,1110.0,...,16096.883087,16485.841727,15213.536147,14218.739547,13917.570567,532.499464,11047.344357,54.406502,46.848373,1198.986182


In [50]:
combined_imbalance_X = df_api_new_merged3[["imbalance_price_predictions_MLP", "national_demand", "transmission_system_demand", "margin"]]
with open("scaler_MLP_extension_imbalance.pkl", "rb") as file:
    scaler = pickle.load(file)
combined_imbalance_X_scaled = scaler.transform(combined_imbalance_X.values)
combined_imbalance_X_scaled = torch.tensor(combined_imbalance_X_scaled, dtype=torch.float32)

model_imbalance = SimpleModel(input_dim=4)
model_imbalance.load_state_dict(torch.load("MLP_extension_imbalance.pth"))
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(combined_imbalance_X_scaled)
predictions = predictions.numpy()
df_api_new_merged3["imbalance_price_predictions_MLP_extension"] = predictions


c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\paulh\AppData\Local\Temp\ipykernel_11040\2423680995.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where

In [51]:
combined_day_ahead_X = df_api_new_merged3[["day_ahead_price_predictions_MLP", "national_demand", "transmission_system_demand", "margin"]]
with open("scaler_MLP_extension_day_ahead.pkl", "rb") as file:
    scaler = pickle.load(file)
combined_day_ahead_X_scaled = scaler.transform(combined_day_ahead_X.values)
combined_day_ahead_X_scaled = torch.tensor(combined_day_ahead_X_scaled, dtype=torch.float32)

model_day_ahead = SimpleModel(input_dim=4)
model_day_ahead.load_state_dict(torch.load("MLP_extension_day_ahead.pth"))
model_day_ahead.eval()
with torch.no_grad():
    predictions = model_day_ahead(combined_day_ahead_X_scaled)
predictions = predictions.numpy()
df_api_new_merged3["day_ahead_price_predictions_MLP_extension"] = predictions

c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\paulh\AppData\Local\Temp\ipykernel_11040\952351878.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where 

In [52]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions_MLP_extension']
    Target_MW = row['5']
    imbalance_price = row['imbalance_price_predictions_MLP_extension']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged3['optimized_trade_MLP'] = df_api_new_merged3.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged3['revenue_normal'] = df_api_new_merged3['day_ahead_price'] * df_api_new_merged3['optimized_trade_MLP'] + \
                         (df_api_new_merged3['Total_MW'] - df_api_new_merged3['optimized_trade_MLP']) * \
                         (df_api_new_merged3['imbalance_price'] - 0.07 * (df_api_new_merged3['Total_MW'] - df_api_new_merged3['optimized_trade_MLP']))


In [53]:
df_api_new_merged3.revenue_normal.mean()

12116.758136159322

In [54]:
df_api_new_merged3.columns

Index(['timestamp_utc', 'market_price', 'day_ahead_price', 'volume',
       'settlement_period_x', 'cos_hour', 'cos_day', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'imbalance_price', 'market_price_lag96h',
       'imbalance_price_lag96h', 'day_ahead_price_lag1week', 'volume_lag96h',
       'generation_solar', 'installed_capacity_mwp', 'capacity_mwp',
       'settlement_date_x', 'settlement_period_y', 'boa', 'generation_wind',
       'imvalance_price_predictions', 'day_ahead_price_predictions',
       'Total_MW', 'Revenue_40', 'Revenue_30', 'Revenue_20', 'Revenue_10',
       'Revenue_60', 'Revenue_50', 'Revenue_70', 'Revenue_80', 'Revenue_90',
       'optimized_trade', 'revenue_normal', 'day_ahead_price_predictions_MLP',
       'imbalance_price_predictions_MLP', 'optimized_trade_MLP',
       'settlement_date_y', 'settlement_period', 'boundary',
       'publish_time_utc_x', 'transmission_system_demand', 'national_demand',
       'date', 'forecast_date', 'publish_time_utc_y', 'm

## newer LSTMs

In [55]:
import torch
import torch.nn as nn

class LSTMPredictor1(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor1, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [56]:
X_imbalance = df_api_new_merged2.dropna()[["imbalance_price"]].dropna().values
# X_imbalance = X_imbalance[384:]

# # Define the sequence length
# sequence_length = 192

# # Function to create sequences from just X (no need for y)
# def create_sequences_from_X(X, sequence_length):
#     sequences = []
#     for i in range(len(X) - sequence_length):
#         # Create a sequence of the desired length
#         seq = X[i:i + sequence_length]
#         sequences.append(seq)
#     return np.array(sequences)

# # Create sequences from X
# X_imbalance = create_sequences_from_X(X_imbalance, sequence_length)

# # Output the shape of the resulting sequences
# print("X_seq shape:", X_imbalance.shape)


In [57]:
X_day_ahead = df_api_new_merged2.dropna()[["day_ahead_price"]].dropna().values
# X_day_ahead = X_day_ahead[384:]

# Define the sequence length
# sequence_length = 192

# # Function to create sequences from just X (no need for y)
# def create_sequences_from_X(X, sequence_length):
#     sequences = []
#     for i in range(len(X) - sequence_length):
#         # Create a sequence of the desired length
#         seq = X[i:i + sequence_length]
#         sequences.append(seq)
#     return np.array(sequences)

# # Create sequences from X
# X_day_ahead = create_sequences_from_X(X_day_ahead, sequence_length)

# # Output the shape of the resulting sequences
# print("X_seq shape:", X_day_ahead.shape)

In [58]:
X_day_ahead.shape

(381, 1)

In [59]:
model_day_ahead_price = LSTMPredictor1(input_size=1, hidden_size=64, num_layers=3, output_size=1, dropout=0.1)
model_day_ahead_price.load_state_dict(torch.load("LSTM_day_ahead_price_only_1Feature.pth"))
model_day_ahead_price.eval()
with torch.no_grad():
    predictions = model_day_ahead_price(torch.tensor(X_day_ahead, dtype=torch.float32).unsqueeze(2))
predictions_dayahead = predictions.numpy()


C:\Users\paulh\AppData\Local\Temp\ipykernel_11040\339041631.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_day_ahead_price.load_state_dict(torch.load("LSTM_day_ah

In [60]:
predictions.shape

torch.Size([381, 1])

In [61]:
df_api2 = df_api_new_merged2.dropna().copy()
df_api2["day_ahead_price_predictions_LSTM"] = predictions_dayahead

In [62]:
model_imbalance_price = LSTMPredictor1(input_size=1, hidden_size=64, num_layers=3, output_size=1, dropout=0.1)
model_imbalance_price.load_state_dict(torch.load("LSTM_imbalance_price_only_1Feature.pth"))
model_imbalance_price.eval()
with torch.no_grad():
    predictions = model_imbalance_price(torch.tensor(X_imbalance, dtype=torch.float32).unsqueeze(2))
predictions_imbalance = predictions.numpy()

df_api2["imbalance_price_predictions_LSTM"] = predictions_imbalance

C:\Users\paulh\AppData\Local\Temp\ipykernel_11040\1974065888.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance_price.load_state_dict(torch.load("LSTM_imbal

In [63]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions_LSTM']
    Target_MW = row['5']
    imbalance_price = row['imbalance_price_predictions_LSTM']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api2['optimized_trade_MLP'] = df_api2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api2['revenue_normal'] = df_api2['day_ahead_price'] * df_api2['optimized_trade_MLP'] + \
                         (df_api2['Total_MW'] - df_api2['optimized_trade_MLP']) * \
                         (df_api2['imbalance_price'] - 0.07 * (df_api2['Total_MW'] - df_api2['optimized_trade_MLP']))


In [64]:
df_api2.revenue_normal.mean()

12449.748795248075